<a href="https://colab.research.google.com/github/Regina-Arthur/YOLOv1_From_Scratch/blob/main/YOLOv1_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Building a YOLOv1 model from scratch by Regina Arthur


###This will be done without the convolutional layers being pretrained on imagenet

##Import the necessary libraries


In [1]:
#Let's import the necessary libraries to build YOLOv1
#with pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms.v2 as v2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
torch.__version__

'2.8.0+cu126'

##Connect To a GPU if available

In [2]:
#Let's check if cuda is available and connect to it.
#If it is not available, use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

##Load the Semantic Boundaries Dataset


### Create a folder to store the Semantic Boundaries Dataset

In [ ]:
#Let's import the necessary libraries for creating a folder in google drive
from google.colab import drive
import os

#Let's mount Google Drive
drive.mount('/content/drive')

def determinedirectory(directory):
  #Let's define folder path in Google Drive
  location = directory.upper()
  root ='/content/drive/MyDrive/sb_Dataset/'
  sb_root = root + location
  #Let's check if the folder exists, if not, let's create it
  if not os.path.exists(sb_root):
      os.makedirs(sb_root)
      print(f"Created new folder: {sb_root}")
  else:
      print(f"Folder already exists: {sb_root}")
  return sb_root


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##How to load the Semantic Boundaries Dataset

Documentation code

**class**

**classtorchvision.datasets.SBDataset(root: Union[str, Path], image_set: str = 'train', mode: str = 'boundaries', download: bool = False, transforms: Optional[Callable] = None)[source]**


In [ ]:
#Let's reshape the images
transforms = v2.Compose([
    v2.Resize((448,448)),
    v2.ToDtype(torch.float32, scale=True),
    ])

In [ ]:
from torchvision.datasets import SBDataset

In [ ]:
TrainData = torchvision.datasets.SBDataset(
    root = determinedirectory("train"),
    image_set = 'train_noval',
    mode = 'boundaries',
    download = True,
    transforms = transforms,
    )

Created new folder: /content/drive/MyDrive/sb_Dataset/TRAIN


100%|██████████| 1.42G/1.42G [00:43<00:00, 32.8MB/s]
100%|██████████| 67.5k/67.5k [00:00<00:00, 1.52MB/s]


##Neural Network Architecture


###I am following the documentation on the MODULE class to build my neural network. All neural network model are to inherit from the nn.Module class.

####Conv2d
####class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

####class torch.nn.LeakyReLU(negative_slope=0.01, inplace=False)

####MaxPool2d
####class torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)[source]

####Linear
####class torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)[source]

####Dropout2d
####class torch.nn.Dropout2d(p=0.5, inplace=False)[source]



In [5]:
class YOLOv1(nn.Module):
  def __init__(self, S=7, B=2, C=20):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels = 3,out_channels = 64, kernel_size = 7,stride = 2, padding = 3)
    self.conv2 = nn.Conv2d(in_channels = 64,out_channels = 192, kernel_size = 3, stride = 1, padding = 1)
    self.conv3 = nn.Conv2d(in_channels = 192,out_channels = 128, kernel_size = 1,stride = 1, padding = 0)
    self.conv4 = nn.Conv2d(in_channels = 128,out_channels = 256, kernel_size = 3, stride = 1, padding = 1)
    self.conv5 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 1, stride = 1, padding = 0)
    self.conv6 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 1)
    self.conv7 = nn.Conv2d(in_channels = 512, out_channels = 256, kernel_size = 1,stride = 1, padding = 0)
    self.conv8 = nn.Conv2d(in_channels = 256,out_channels = 512, kernel_size = 3, stride = 1, padding = 1)
    self.conv9 = nn.Conv2d(in_channels = 512,out_channels = 256, kernel_size = 1,stride = 1, padding = 0)
    self.conv10 = nn.Conv2d(in_channels = 256,out_channels = 512, kernel_size = 3,stride = 1, padding =1)
    self.conv11 = nn.Conv2d(in_channels = 512,out_channels = 256, kernel_size = 1,stride = 1, padding = 0)
    self.conv12 = nn.Conv2d(in_channels = 256,out_channels = 512, kernel_size = 3,stride = 1, padding = 1)
    self.conv13 = nn.Conv2d(in_channels = 512,out_channels = 256, kernel_size = 1,stride = 1, padding = 0)
    self.conv14 = nn.Conv2d(in_channels = 256,out_channels = 512, kernel_size = 3,stride = 1, padding = 1)
    self.conv15 = nn.Conv2d(in_channels = 512,out_channels = 512, kernel_size = 1,stride = 1, padding = 0)
    self.conv16 = nn.Conv2d(in_channels = 512,out_channels = 1024, kernel_size = 3,stride = 1, padding = 1)
    self.conv17 = nn.Conv2d(in_channels = 1024,out_channels = 512, kernel_size = 1,stride = 1, padding = 0)
    self.conv18 = nn.Conv2d(in_channels = 512,out_channels = 1024, kernel_size = 3,stride = 1, padding = 1)
    self.conv19 = nn.Conv2d(in_channels = 1024,out_channels = 512, kernel_size = 1,stride = 1, padding = 0)
    self.conv20 = nn.Conv2d(in_channels = 512,out_channels = 1024, kernel_size = 3,stride = 1, padding = 1)
    self.conv21 = nn.Conv2d(in_channels = 1024,out_channels = 1024, kernel_size = 3,stride = 1, padding = 1)
    self.conv22 = nn.Conv2d(in_channels = 1024,out_channels = 1024, kernel_size = 3,stride = 2, padding = 1)
    self.conv23 = nn.Conv2d(in_channels = 1024,out_channels = 1024, kernel_size = 3,stride = 1, padding = 1)
    self.conv24 = nn.Conv2d(in_channels = 1024,out_channels = 1024, kernel_size = 3,stride = 1, padding = 1)

    #Pooling and Activation
    self.MaxPooling2d = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.LeakyReLU = nn.LeakyReLU(negative_slope = 0.1)

    #Flattening and Fully Connected Layer
    self.Flatten = nn.Flatten()
    self.Linear = nn.Linear(in_features = 50176 , out_features = 4096)
    self.Dropout = nn.Dropout2d()
    self.Linear2 = nn.Linear(in_features =4096 , out_features = S * S *( B * 5 + 20))

  def forward(self, x):
    x = self.MaxPooling2d(self.LeakyReLU(self.conv1(x)))
    x = self.MaxPooling2d(self.LeakyReLU(self.conv2(x)))
    x = self.LeakyReLU(self.conv3(x))
    x = self.LeakyReLU(self.conv4(x))
    x = self.LeakyReLU(self.conv5(x))
    x = self.MaxPooling2d(self.LeakyReLU(self.conv6(x)))
    x = self.LeakyReLU(self.conv7(x))
    x = self.LeakyReLU(self.conv8(x))
    x = self.LeakyReLU(self.conv9(x))
    x = self.LeakyReLU(self.conv10(x))
    x = self.LeakyReLU(self.conv11(x))
    x = self.LeakyReLU(self.conv12(x))
    x = self.LeakyReLU(self.conv13(x))
    x = self.LeakyReLU(self.conv14(x))
    x = self.LeakyReLU(self.conv15(x))
    x = self.MaxPooling2d(self.LeakyReLU(self.conv16(x)))
    x = self.LeakyReLU(self.conv17(x))
    x = self.LeakyReLU(self.conv18(x))
    x = self.LeakyReLU(self.conv19(x))
    x = self.LeakyReLU(self.conv20(x))
    x = self.LeakyReLU(self.conv21(x))
    x = self.LeakyReLU(self.conv22(x))
    x = self.LeakyReLU(self.conv23(x))
    x = self.LeakyReLU(self.conv24(x))
    x = self.LeakyReLU(self.Linear(self.Flatten(x)))
    x = self.Dropout(x)
    x = self.Linear2(x)
    return x



## Architecture Test
This just makes sure the architecture works as intended


In [6]:
model = YOLOv1()
model = model.to(device)

In [7]:
dummy_input = torch.randn(1, 3, 448, 448).to(device)

with torch.no_grad():
    output = model(dummy_input)

print("Output shape:", output.shape)

Output shape: torch.Size([1, 1470])


/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:1535: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [8]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total params: {total_params:,}")
print(f"Trainable params: {trainable_params:,}")

Total params: 271,703,550
Trainable params: 271,703,550


In [ ]:
print(YOLOv1().view)

In [ ]:
torch.manual_seed(0)

##Model loss

In [ ]:
class YOLOv1Loss(nn.Module):
  def __init__(self, S=7, B=2, C=20, λ_coord=5, λ_noobj=0.5):
    super().__init__()